In [51]:
%load_ext watermark
%watermark

The watermark extension is already loaded. To reload it, use:
  %reload_ext watermark
2019-04-11T03:45:57-05:00

CPython 3.7.3rc1
IPython 7.3.0

compiler   : MSC v.1916 64 bit (AMD64)
system     : Windows
release    : 10
machine    : AMD64
processor  : Intel64 Family 6 Model 142 Stepping 9, GenuineIntel
CPU cores  : 4
interpreter: 64bit


### Intro a peticiones HTTP con Requests

En esta sección vamos a ver como usar la libreria [`requests`](https://github.com/requests/requests), que es la librería mas comunmente utilizada para hacer peticioes HTTP en python. Se instala desde Anaconda o pip (con `pip install requests`)

In [52]:
import requests

### GET Requests

Cuando queremos obtener algo de una página web (un *recurso* como se llaman de forma más técnica), lo que nuestro navegador hace es una petición HTTP de tipo **GET**. 

Por ejemplo, si queremos hacer una petición a meneame.net lo hacemos de forma sencilla de esta forma:

In [53]:
respuesta = requests.get("http://mexitel.sre.gob.mx")

Podemos verificar que la petición ha tenido exito con el atributo `ok`

In [54]:
respuesta.ok

True

Y podemos ver el status de la respuesta con `status_code`

In [55]:
respuesta.status_code

200

En este caso es un **200** por que la petición ha sido recibida con éxito

podemos ver el contenido que nos ha enviado el servidor con `content`

In [56]:
print(respuesta.content)

b'<?xml version="1.0" encoding="UTF-8" ?>\n<!DOCTYPE html>\n<html xmlns="http://www.w3.org/1999/xhtml"><head><link type="text/css" rel="stylesheet" href="/citas.webportal/javax.faces.resource/theme.css.jsf?ln=primefaces-sregobmx" /><link type="text/css" rel="stylesheet" href="/citas.webportal/javax.faces.resource/fa/font-awesome.css.jsf?ln=primefaces&amp;v=5.2" /><link type="text/css" rel="stylesheet" href="/citas.webportal/javax.faces.resource/citas.webGobMx.css.jsf?ln=css" /><script type="text/javascript" src="/citas.webportal/javax.faces.resource/jquery/jquery.js.jsf?ln=primefaces&amp;v=5.2"></script><script type="text/javascript" src="/citas.webportal/javax.faces.resource/primefaces.js.jsf?ln=primefaces&amp;v=5.2"></script><link type="text/css" rel="stylesheet" href="/citas.webportal/javax.faces.resource/primefaces.css.jsf?ln=primefaces&amp;v=5.2" /><link type="text/css" rel="stylesheet" href="/citas.webportal/javax.faces.resource/watermark/watermark.css.jsf?ln=primefaces&amp;v=5.2

Vemos que la petición a google nos ha devuelto el html de la página de inicio de google. Podemos escribirlo a un archivo y verlo

In [57]:
with open("meneame.html", "wb") as fname:
    fname.write(respuesta.content)

In [58]:
import webbrowser
webbrowser.open("meneame.html")

True

Vemos que la página que hemos obtenido via `requests` no es la misma que la que veríamos desde el navegador. Esto es por que `requests` no sabe como interpretar javascript y dicha página genera los contenidos de forma dinámica.

### Reddit

Supongamos ahora que queremos obtener los 5 primeros posts del foro de comida en Reddit. Podemos hacer una petición a www.reddit.com/r/food (que es la misma url que usariamos en el navegador).

In [59]:
respuesta = requests.get("http://www.reddit.com/r/food")

podemos verificar que la peticion ha sido correcta comprobando el status de la respuesta:

In [60]:
respuesta.ok

False

Oh que sorpresa! La petición no ha funcionado, veamos que status hemos recibido.

In [61]:
respuesta.status_code

429

Sabemos que los códigos http que empiezan por *4xx* significan errores en la petición que hemos hecho. Podemos mirar lo que significa el código 429 en una [lista](https://developer.mozilla.org/en-US/docs/Web/HTTP/Headers/User-Agent), o podemos mirar la razon del error con el atributo `reason`

In [62]:
respuesta.reason

'Too Many Requests'

Sin embargo, si abrimos esa dirección web (url) en el navegador funciona sin problemas.

In [63]:
webbrowser.open("www.reddit.com/r/food")

True

Vaya! Nos dice el servidor de reddit que hemos hecho demasiadas peticiones, ¿como puede ser si solo hemos hecho una?. Hemos mencionado que ciertas páginas web tienen un conjunto de contramedidas para evitar bots (programas que pretenden ser usuarios).

Por este motivo, podemos hacer uso de la api que reddit nos proporciona, como dicha api está diseñada para ser consumida por bots, no tendremos problema.

In [34]:
respuesta = requests.get("https://www.reddit.com/r/food/.json?limit=5")

In [35]:
respuesta.ok

False

In [36]:
print(respuesta.content)

b'{"message": "Too Many Requests", "error": 429}'


la petición de la api viene en formato json, con requests podemos parsearla facilmente:

In [27]:
datos = respuesta.json()

In [28]:
from pprint import pprint

In [29]:
pprint(datos)

{'error': 429, 'message': 'Too Many Requests'}


Vemos que datos es un diccionario con la clave `data` y subclave `children` que tiene los posts en una lista. Dentro de cada lista, la clave `title` tiene el titulo del post. Por ejemplo, si queremos el título del segundo post en la lista:

In [30]:
datos['data']['children'][1]['data']['title']

KeyError: 'data'

Asi que si queremos todos los titulos no tenemos más que iterar los posts:

In [27]:
titulos = []
for post in datos['data']['children']:
    titulos.append(post['data']['title'])
    
titulos

['[MOD POST - PSA **UPDATE**] pLEASE SEE THE FOLLOWING RULE CHANGES FOR POSTING LINKS IN THIS SUB.',
 '[homemade] chocolate cupcakes with macarones, Italian buttercream, meringue kisses and roses.',
 '[I Ate] Lemon Meringue Pie',
 '[Homemade] One Dozen Chocolate Chip Cookies',
 '[Homemade] Carbonara',
 '[I Ate] Deep Fried Cheese']

Alternativamente, podemos usar [`glom`](https://github.com/mahmoud/glom) para iterar más rápidamente. **NOTA:** `glom` es una libreria nueva y puede no funcionar de forma estable

In [28]:
from glom import glom
glom(datos, ('data.children', ['data.title']))

['[MOD POST - PSA **UPDATE**] pLEASE SEE THE FOLLOWING RULE CHANGES FOR POSTING LINKS IN THIS SUB.',
 '[homemade] chocolate cupcakes with macarones, Italian buttercream, meringue kisses and roses.',
 '[I Ate] Lemon Meringue Pie',
 '[Homemade] One Dozen Chocolate Chip Cookies',
 '[Homemade] Carbonara',
 '[I Ate] Deep Fried Cheese']

### Yahoo Weather

Podemos usar la [api de Yahoo weather](https://developer.yahoo.com/weather/?guccounter=2) de forma similar a la de reddit

In [64]:
url = """
https://query.yahooapis.com/v1/public/yql?q=select * from weather.forecast where woeid in (select woeid from geo.places(1) where text="{}") and u='c'&format=json
"""

url

'\nhttps://query.yahooapis.com/v1/public/yql?q=select * from weather.forecast where woeid in (select woeid from geo.places(1) where text="{}") and u=\'c\'&format=json\n'

In [65]:
url.format("Murcia, Spain")

'\nhttps://query.yahooapis.com/v1/public/yql?q=select * from weather.forecast where woeid in (select woeid from geo.places(1) where text="Murcia, Spain") and u=\'c\'&format=json\n'

In [66]:
datos = requests.get(url.format("Madrid, Spain")).json()

ConnectionError: HTTPSConnectionPool(host='query.yahooapis.com', port=443): Max retries exceeded with url: /v1/public/yql?q=select%20*%20from%20weather.forecast%20where%20woeid%20in%20(select%20woeid%20from%20geo.places(1)%20where%20text=%22Madrid,%20Spain%22)%20and%20u='c'&format=json%0A (Caused by NewConnectionError('<urllib3.connection.VerifiedHTTPSConnection object at 0x000002AFB0228278>: Failed to establish a new connection: [Errno 11001] getaddrinfo failed'))

In [44]:
from pprint import pprint
pprint(datos)

{'error': 429, 'message': 'Too Many Requests'}


In [45]:
from glom import glom
temperaturas = glom(datos, {
    "date": ('query.results.channel.item.forecast', ['date']),
    "high":  ('query.results.channel.item.forecast', ['high']),
    "low":  ('query.results.channel.item.forecast', ['low'])
})

ModuleNotFoundError: No module named 'glom'

In [36]:
temperaturas

{'date': ['14 May 2018',
  '15 May 2018',
  '16 May 2018',
  '17 May 2018',
  '18 May 2018',
  '19 May 2018',
  '20 May 2018',
  '21 May 2018',
  '22 May 2018',
  '23 May 2018'],
 'high': ['19', '23', '25', '23', '22', '23', '26', '24', '22', '26'],
 'low': ['10', '8', '9', '11', '12', '11', '11', '12', '12', '12']}

In [46]:
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

plt.rcParams["figure.figsize"] = (7, 7)

In [47]:
plt.style.use("ggplot")

In [48]:
plt.plot(range(len(temperaturas["date"])), temperaturas["high"], 
         label="Máxima")

plt.plot(range(len(temperaturas["date"])), temperaturas["low"], 
         label="Mínima")
plt.xticks(range(len(temperaturas["date"])), temperaturas["date"],
           rotation=45, ha="right")
plt.title("Pronóstico de temperatura para Madrid");
plt.legend();

NameError: name 'temperaturas' is not defined

In [49]:
def dibujar_grafico_temperaturas(temperaturas, ciudad):
    plt.plot(range(len(temperaturas["date"])), temperaturas["high"], 
         label="Máxima")
    plt.plot(range(len(temperaturas["date"])), temperaturas["low"], 
         label="Mínima")
    plt.xticks(range(len(temperaturas["date"])), temperaturas["date"],
           rotation=45, ha="right")
    plt.legend()
    plt.title("Pronóstico de temperatura para {}".format(ciudad));
    
def obtener_temperaturas(ciudad):
    datos = requests.get(url.format(ciudad)).json()
    temperaturas = glom(datos, {
        "date": ('query.results.channel.item.forecast', ['date']),
        "high":  ('query.results.channel.item.forecast', ['high']),
        "low":  ('query.results.channel.item.forecast', ['low'])
    })
    return temperaturas

def grafico_temperaturas(ciudad):
    temperaturas = obtener_temperaturas(ciudad)
    dibujar_grafico_temperaturas(temperaturas, ciudad)

In [50]:
grafico_temperaturas("New York City")

ConnectionError: HTTPSConnectionPool(host='query.yahooapis.com', port=443): Max retries exceeded with url: /v1/public/yql?q=select%20*%20from%20weather.forecast%20where%20woeid%20in%20(select%20woeid%20from%20geo.places(1)%20where%20text=%22New%20York%20City%22)%20and%20u='c'&format=json%0A (Caused by NewConnectionError('<urllib3.connection.VerifiedHTTPSConnection object at 0x000002AFA6A57DD8>: Failed to establish a new connection: [Errno 11001] getaddrinfo failed'))